In [11]:
import time
import numpy as np

import os.path as op

from copy import deepcopy

from tqdm import tqdm
import matplotlib.pyplot as plt

import h5py

import nibabel as nib
import pandas as pd

import sys
sys.path.append("../")

from src import power_crtl
from src import pkl_custom
from src import plot_stat_on_surf

In [4]:
def standardize_array(arr):
    """
    Standardizes a numpy array to a range from 0 to 1.
    
    Parameters:
    arr (numpy.ndarray): Input array to be standardized.
    
    Returns:
    numpy.ndarray: Standardized array with values ranging from 0 to 1.
    """
    min_val = np.min(arr)
    max_val = np.max(arr)
    standardized_arr = (arr - min_val) / (max_val - min_val)
    return standardized_arr

def flatten_distribution(arr):
    """
    Flattens the distribution of a numpy array while keeping the original order.
    
    Parameters:
    arr (numpy.ndarray): Input array to be flattened.
    
    Returns:
    numpy.ndarray: Flattened array with the same order as the input.
    """
    sorted_indices = np.argsort(arr)
    sorted_arr = np.sort(arr)
    flattened_arr = np.linspace(sorted_arr[0], sorted_arr[-1], len(arr))
    result = np.empty_like(arr)
    result[sorted_indices] = flattened_arr
    return result

def gaussianize_distribution(arr):
    """
    Projects a distribution of a numpy array into a Gaussian distributed array while keeping the original order.
    
    Parameters:
    arr (numpy.ndarray): Input array to be transformed.
    
    Returns:
    numpy.ndarray: Gaussianized array with the same order as the input.
    """
    sorted_indices = np.argsort(arr)
    gaussian_arr = np.random.normal(loc=0, scale=1, size=len(arr))
    gaussian_arr.sort()
    result = np.empty_like(arr)
    result[sorted_indices] = gaussian_arr
    return result

def even_distribution(distrib, factor=2):
    if factor == -1:
        return gaussianize_distribution(distrib)
    if factor == 0:
        return flatten_distribution(distrib)
    amp = np.abs(distrib)
    ret = np.sign(distrib) * amp ** (1/factor)
    return ret

In [8]:
root_path = "../"
data_path = "/Users/mikichan/Desktop/mywork/atlas_data/"
data_path_pet = "/Users/mikichan/Desktop/mywork/atlas_data/PET/FDG_PET"

In [9]:
scale = 1
connFilename = op.join(data_path, f'fiber_atlas/probconnatlas/wm.connatlas.scale{scale}.h5')

hf = h5py.File(connFilename, 'r')
centers = np.array(hf.get('header').get('gmcoords'))
gmregions_names = hf.get('header').get('gmregions')[()]
nb_regions = gmregions_names.shape[0]
hf.close()

roi_path = op.join(data_path, 'fiber_atlas/gmlabels_vol.nii')
roi_atlas = nib.load(roi_path)
atlas_data, affine = roi_atlas.get_fdata(), roi_atlas.affine

In [17]:
list(pd.read_csv(op.join(data_path_pet, 'ds003397/sub-01/sub-01_scans.tsv')).iloc[0])

['anat/sub-01_run-1_T1w.nii.gz\t2018-08-24T11:49:06\tn/a\t1a23f124']

In [10]:
op.join(data_path_pet, 'ds003397/sub-01/pet'/)

'/Users/mikichan/Desktop/mywork/atlas_data/PET/FDG_PET/ds003397/sub-01/pet'

In [11]:
rstfMRI = nib.load(op.join(data_path, 'HCP_sample/hcp_smoothen_group_fmri.nii'))
rst_timecourses = rstfMRI.get_fdata()

if op.exists(op.join(root_path, 'resources/hcp-rst_lausanne2018.pkl')):
    roi_timecourses = pkl_custom.open_pkl_file(op.join(root_path, 'resources/hcp-rst_lausanne2018.pkl'))
else:
    roi_timecourses = np.zeros((len(gmregions_names), rst_timecourses.shape[-1]))
    for k in range(1, len(gmregions_names) + 1):
        roi_timecourses[k-1] = rst_timecourses[atlas_data == k, :].mean(axis=0)
    pkl_custom.save_pkl_file(op.join(root_path, 'resources/hcp-rst_lausanne2018.pkl'), roi_timecourses)

In [ ]:
# plot_stat_on_surf.plot_node_signal_on_surf(even_distribution(mean_distribution, factor=1),
#                                            roi_path, gmregions_names, vmin=-0.2, vmax=1.2, cmap='gnuplot2')